In [1]:
import json
from enum import Enum
from collections import namedtuple

In [37]:
class events_list(Enum): 
    Write = 0
    Read = 1
    Acquire = 2
    Release = 3

In [3]:
def _json_object_hook(d): 
    return namedtuple('event_raw', d.keys())(*d.values())

def json2obj(data): 
    return json.loads(data, object_hook=_json_object_hook)

class event_raw:
    def __init__(self):
        self.tid = -1
        self.e_type = 0
        self.var_name = ""
        self.vc = []
        self.vc_r = []
        self.vc_w = []

In [38]:
class critical_section:
    def __init__(self,thread_num):
        self.locks = set()
        self.read_objects = set()
        self.write_objects = set()
        self.child_sections = []
        self.tid = thread_num
        self.event_start = -1
        self.event_end = -1

    def __str__(self):
        ret_str = "Thread : %d \nLocks : %s \nRead Objects : %s \nWrite Objects : %s \nStart : %d \nEnd : %d \nChild Sections :\n" % (self.tid,self.locks,self.read_objects,self.write_objects,self.event_start,self.event_end)
        for child_cs in self.child_sections:
            ret_str= ret_str+str(child_cs)
        return ret_str
    
    def clear_lists(self):
        self.locks.clear()
        self.read_objects.clear()
        self.write_objects.clear()
        self.child_sections.clear()
    
    def create_child_Section(self,evt_obj,thrd_stk,evt_id):
        temp = self
        size_stk = len(thrd_stk)
        while size_stk-1 > 1:
            if len(temp.child_sections) > 0:
                temp.child_sections[-1].locks.add(evt_obj.var_name)
                temp = temp.child_sections[-1]
            size_stk=size_stk-1
        temp.child_sections.append(critical_section(evt_obj.tid))
        temp.child_sections[-1].locks.add(evt_obj.var_name)
        temp.child_sections[-1].event_start = evt_id
    
    def operate_child(self,evt_obj,thrd_stk):
        temp = self
        size_stk = len(thrd_stk)
        while size_stk-1 > 0:
            temp = temp.child_sections[-1]
            size_stk=size_stk-1
            temp.add_object(evt_obj)
#             if evt_obj.e_type == events_list.Read.value:
#                 temp.read_objects.add(evt_obj.var_name)
#             elif if evt_obj.e_type == events_list.Write.value:
#                 temp.write_objects.add(evt_obj.var_name)
    
    def add_object(self,evt_obj):
        if evt_obj.e_type == events_list.Read.value:
                self.read_objects.add(evt_obj.var_name)
        elif evt_obj.e_type == events_list.Write.value:
                self.write_objects.add(evt_obj.var_name)
    
    def update_event_end(self,end_id,thrd_stk):
        temp = self
        size_stk = len(thrd_stk)
        while size_stk-1 > 0:
            temp = temp.child_sections[-1]
            size_stk=size_stk-1
        temp.event_end = end_id

In [43]:
# event_log_raw = open("../Figure1.log")
event_log_raw = open("temp.log")
# event_log_raw = open("../Test.log")
event_log = json.loads(event_log_raw.read())

In [44]:
events = []
for line in event_log:
    events.append(json2obj(json.dumps(line)))

In [47]:
thread_cs = []
critical_section_list = []
thread_lock_stack = []
ct = 0
for x in events:
    if(len(thread_cs)<x.tid):
        while(len(thread_cs)!=x.tid):
            thread_cs.append(None)
            thread_lock_stack.append([])
    if thread_cs[x.tid-1]==None: # Start of new crititcal section
        print("Starting new critial section : ")
        thread_cs[x.tid-1] = critical_section(x.tid)
        thread_cs[x.tid-1].clear_lists()
        thread_cs[x.tid-1].event_start = ct
        if x.e_type == events_list.Read.value or x.e_type == events_list.Write.value:
#             print("Here")
            thread_cs[x.tid-1].add_object(x)#.objects.add(x.var_name)
            thread_cs[x.tid-1].event_end = ct
            critical_section_list.append(thread_cs[x.tid-1])
            thread_cs[x.tid-1] = None
        elif x.e_type == events_list.Acquire.value:
            thread_cs[x.tid-1].locks.add(x.var_name)
            thread_lock_stack[x.tid-1].append(x.var_name)
    else:
        if x.e_type == events_list.Read.value or x.e_type == events_list.Write.value:
            if len(thread_lock_stack[x.tid-1]) > 1:
                thread_cs[x.tid-1].operate_child(x,thread_lock_stack[x.tid-1])
            thread_cs[x.tid-1].add_object(x)#.objects.add(x.var_name)
        elif x.e_type == events_list.Acquire.value:
            if len(thread_lock_stack[x.tid-1]) == 0:
                thread_cs[x.tid-1].event_end = ct-1
                critical_section_list.append(thread_cs[x.tid-1])
                thread_cs[x.tid-1] = None
                thread_cs[x.tid-1] = critical_section(x.tid)
                thread_cs[x.tid-1].clear_lists()
                thread_cs[x.tid-1].event_start = ct
                thread_lock_stack[x.tid-1].append(x.var_name)
            elif thread_lock_stack[x.tid-1][-1] != x.var_name:
                thread_lock_stack[x.tid-1].append(x.var_name)
                thread_cs[x.tid-1].create_child_Section(x,thread_lock_stack[x.tid-1],ct)
            thread_cs[x.tid-1].locks.add(x.var_name)
        elif x.e_type == events_list.Release.value:
            thread_cs[x.tid-1].update_event_end(ct,thread_lock_stack[x.tid-1])
            if len(thread_lock_stack[x.tid-1]) == 1:
#                 print(thread_cs[x.tid-1])
                critical_section_list.append(thread_cs[x.tid-1])
                thread_cs[x.tid-1] = None
            # Note: Optimistic assumption, top of the lock is same
            if thread_lock_stack[x.tid-1][-1] == x.var_name:
#                 print("Lock Stack before pop : "+str(thread_lock_stack[x.tid-1])+"\n")
                thread_lock_stack[x.tid-1].pop()
#     print(thread_cs[x.tid-1])
    ct+=1

Starting new critial section : 
None
Starting new critial section : 
None
Starting new critial section : 
Thread : 1 
Locks : set() 
Read Objects : set() 
Write Objects : set() 
Start : 2 
End : -1 
Child Sections :

Thread : 1 
Locks : set() 
Read Objects : {'a'} 
Write Objects : set() 
Start : 2 
End : -1 
Child Sections :

Thread : 1 
Locks : {'b'} 
Read Objects : set() 
Write Objects : set() 
Start : 4 
End : -1 
Child Sections :

None
Starting new critial section : 
None
Starting new critial section : 
Thread : 1 
Locks : {'y'} 
Read Objects : set() 
Write Objects : set() 
Start : 7 
End : -1 
Child Sections :



IndexError: list index out of range

In [42]:
for cs in critical_section_list:
    print(cs)

Thread : 1 
Locks : set() 
Read Objects : set() 
Write Objects : {'test/Test_y'} 
Start : 0 
End : 0 
Child Sections :

Thread : 1 
Locks : set() 
Read Objects : {'test/Test_y'} 
Write Objects : set() 
Start : 1 
End : 1 
Child Sections :

Thread : 1 
Locks : set() 
Read Objects : set() 
Write Objects : {'test/Test_y'} 
Start : 2 
End : 2 
Child Sections :

Thread : 1 
Locks : set() 
Read Objects : {'test/Test_y'} 
Write Objects : set() 
Start : 3 
End : 3 
Child Sections :

Thread : 1 
Locks : set() 
Read Objects : set() 
Write Objects : {'test/Test_y'} 
Start : 4 
End : 4 
Child Sections :

Thread : 1 
Locks : set() 
Read Objects : {'test/Test_y'} 
Write Objects : set() 
Start : 5 
End : 5 
Child Sections :

Thread : 1 
Locks : set() 
Read Objects : set() 
Write Objects : {'test/Test_y'} 
Start : 6 
End : 6 
Child Sections :

Thread : 1 
Locks : set() 
Read Objects : {'test/Test_y'} 
Write Objects : set() 
Start : 7 
End : 7 
Child Sections :

Thread : 1 
Locks : set() 
Read Objects 

401